In [7]:
import pandas as pd

# Define the file path
file_path = r"D:\ML\Innomatics_Research_Lab_Internship\task8_SearchEngine\df_cleaned_chunks12-04.csv"

# Define the range of rows to read
start_row = 0
end_row = 150

# Calculate the number of rows to read
rows_to_read = end_row - start_row

# Read rows 0 to 45000 from the CSV file
df_sample = pd.read_csv(file_path, skiprows=start_row, nrows=rows_to_read)

# Display the first few rows of the sampled DataFrame
print(df_sample.head())


       num                                               name  \
0  9180533                         the.message.(1976).eng.1cd   
1  9180583  here.comes.the.grump.s01.e09.joltin.jack.in.bo...   
2  9180592    yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd   
3  9180594    yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd   
4  9180600                              broker.(2022).eng.1cd   

                                   extracted_content  \
0  watch video online open subtitle free browser ...   
1  ah there 's princess dawn terry blooney looney...   
2  yumi 's cell episode extremely polite yumi yum...   
3  watch video online open subtitle free browser ...   
4  watch video online open subtitle free browser ...   

                                     chunked_content  
0  [['watch', 'video', 'online', 'open', 'subtitl...  
1  [['ah', 'there', "'s", 'princess', 'dawn', 'te...  
2  [['yumi', "'s", 'cell', 'episode', 'extremely'...  
3  [['watch', 'video', 'online', 'open', 'subtitl...

# Try above method to get rows from particular rows or we can also try below method to take random rows according to your required percentage.

In [1]:
import pandas as pd
import random

# Define the file path
file_path = r"D:\ML\Innomatics_Research_Lab_Internship\task8_SearchEngine\df_cleaned_chunks12-04.csv"

# Read the total number of rows in the CSV file (you may need to adjust engine='python' if you encounter issues)
total_rows = sum(1 for line in open(file_path))

# Calculate approximately 30% of the total rows
sample_size = int(0.1 * total_rows)

# Randomly select rows from the CSV file
# Skip rows that are not part of the random sample
skip_rows = sorted(random.sample(range(1, total_rows + 1), total_rows - sample_size))

# Read the CSV file, skipping the rows that are not part of the random sample
df_sample = pd.read_csv(file_path, skiprows=skip_rows)

# Display the first few rows of the sampled DataFrame
print(df_sample.head())


       num                                               name  \
0  9181571  trying.s02.e02.the.sun.on.your.back.(2021).eng...   
1  9181886  doctor.lawyer.s01.e14.episode.1.14.(2022).eng.1cd   
2  9181932  alchemy.of.souls.s01.e11.episode.1.11.(2022).e...   
3  9181967    the.governor.s01.e06.episode.1.6.(1995).eng.1cd   
4  9181995  american.experience.s33.e07.the.blinding.of.is...   

                                   extracted_content  \
0  nikki sigh even know we 're i 've already got ...   
1  name people incident background drama fictiona...   
2  enjoy vod high quality tv get live tv movie sh...   
3  cell door clanging braithwaite hey i 'm onto k...   
4  watch video online open subtitle free browser ...   

                                     chunked_content  
0  [['nikki', 'sigh', 'even', 'know', 'we', "'re"...  
1  [['name', 'people', 'incident', 'background', ...  
2  [['enjoy', 'vod', 'high', 'quality', 'tv', 'ge...  
3  [['cell', 'door', 'clanging', 'braithwaite', '...

In [2]:
df=df_sample

In [3]:
df.shape

(8249, 4)

In [11]:
import sqlite3
from sentence_transformers import SentenceTransformer
import json
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Function to persist data to SQLite database
def persist_to_database(database_path, data):
    conn = sqlite3.connect(database_path)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS records
                 (num TEXT, name TEXT, embeddings TEXT)''')
    
    for record in data:
        num = record['num']
        name = record['name']
        embeddings = json.dumps(record['embeddings'])  # Convert embeddings list to JSON string
        c.execute("INSERT INTO records (num, name, embeddings) VALUES (?, ?, ?)", (num, name, embeddings))
    
    conn.commit()
    conn.close()

# Initialize BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')
start_index = 100
end_index = 150
# Example DataFrame with columns: num, name, clean_content
# df1 = df.head(150)
df1=df.iloc[start_index:end_index].copy()

# Chunk size and overlap
chunk_size = 512  # Adjust this according to your needs
overlap_size = 100  # Adjust this according to your needs

# Example data processing, chunking, and encoding
data_to_persist = []
for index, row in df1.iterrows():
    num = row['num']
    name = row['name']
    clean_content = row['extracted_content']
    chunks = [clean_content[i:i+chunk_size] for i in range(0, len(clean_content), chunk_size - overlap_size)]
    for i, chunk in enumerate(chunks):
        embeddings = model.encode([chunk])[0].tolist()
        data_to_persist.append({'num': f"{num}_chunk{i+1}", 'name': name, 'embeddings': embeddings})

# Database path
database_path = 'D:\\ML\\Innomatics_Research_Lab_Internship\\chroma_50.sqlite3'

# Persist data to SQLite database
persist_to_database(database_path, data_to_persist)

# Function to retrieve top 10 similar records based on query
def get_top_10_unique_names(query, database_path):
    conn = sqlite3.connect(database_path)
    c = conn.cursor()
    
    query_embedding = model.encode([query])
    similarities = []
    c.execute("SELECT * FROM records")
    for row in c.fetchall():
        record_num, record_name, record_embeddings = row
        embeddings = json.loads(record_embeddings)
        embeddings = np.array(embeddings).reshape(1, -1)
        similarity = cosine_similarity(query_embedding, embeddings)[0][0]
        similarities.append((record_name, similarity))
    
    # Sort by similarity score and extract unique names
    sorted_names = [name for name, _ in sorted(similarities, key=lambda x: x[1], reverse=True)]
    unique_names = []
    for name in sorted_names:
        if name not in unique_names:
            unique_names.append(name)
            if len(unique_names) == 10:
                break
    
    conn.close()
    return unique_names

# Example usage
query = "   "
top_10_unique_names = get_top_10_unique_names(query, database_path)
print(top_10_unique_names)

['westworld.s04.e05.zhuangzi.(2022).eng.1cd', 'treasure.city.(2020).eng.1cd', 'a.night.in.97.(2020).eng.1cd', 'the.roundup.(2022).eng.1cd', 'rudrabinar.obhishaap.s02.e06.jogphawl.(2022).eng.1cd', 'my.unfamiliar.family.s01.e03.episode.1.3.(2020).eng.1cd', 'the.governor.s01.e05.episode.1.5.(1995).eng.1cd', 'avaete.semente.da.vinganca.(1985).eng.1cd', 'flowers.in.the.attic.the.origin.s01.e03.part.three.the.murderer.(2022).eng.1cd', 'trying.s02.e03.big.heads.(2021).eng.1cd']
